In [1]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np

def get_binance_ohlc(symbol, interval, limit=1000):
    url = f'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    ohlc = []
    for item in data:
        ohlc.append({
            'timestamp': datetime.fromtimestamp(item[0] / 1000),
            'open': float(item[1]),
            'high': float(item[2]),
            'low': float(item[3]),
            'close': float(item[4])
        })
    df = pd.DataFrame(ohlc)
    # df.set_index('timestamp', inplace=True)
    return df



In [2]:
from datetime import datetime, timedelta

def generate_date_list(start_date_str, end_date_str, interval_days=5):
    # Convert string inputs to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    
    # Generate list of dates
    date_list = []
    current_date = start_date
    while current_date <= end_date:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=interval_days)
    
    return date_list

In [3]:
def get_binance_ohlc_time(symbol, interval, start_str, end_str):
    print(start_str,"->",end_str)
    url = f'https://api.binance.com/api/v3/klines'
    
    # Convert start and end times to milliseconds since epoch
    start_ts = int(datetime.strptime(start_str, '%Y-%m-%d %H:%M:%S').timestamp() * 1000)
    end_ts = int(datetime.strptime(end_str, '%Y-%m-%d %H:%M:%S').timestamp() * 1000)
    
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_ts,
        'endTime': end_ts
    }
    response = requests.get(url, params=params)
    data = response.json()
    ohlc = []
    for item in data:
        ohlc.append({
            'timestamp': datetime.fromtimestamp(item[0] / 1000),
            'open': float(item[1]),
            'high': float(item[2]),
            'low': float(item[3]),
            'close': float(item[4])
        })
    
    df = pd.DataFrame(ohlc)
    # df.set_index('timestamp', inplace=True)
    return df

In [4]:
# Example usage
symbol = 'ORDIUSDT'
interval = '15m'  # 1 hour interval
start_str = '2024-06-08 00:00:00'  # Specify your start time
end_str = '2024-07-10 00:00:00'    # Specify your end time

df = get_binance_ohlc_time(symbol, interval, start_str, end_str)
print(df)

2024-06-08 00:00:00 -> 2024-07-10 00:00:00
              timestamp   open   high    low  close
0   2024-06-08 00:00:00  63.72  63.72  62.52  62.66
1   2024-06-08 00:15:00  62.65  63.19  62.25  62.83
2   2024-06-08 00:30:00  62.83  62.95  61.69  62.25
3   2024-06-08 00:45:00  62.25  62.37  61.18  61.55
4   2024-06-08 01:00:00  61.54  61.87  55.90  56.43
..                  ...    ...    ...    ...    ...
495 2024-06-13 03:45:00  51.97  52.04  51.74  51.90
496 2024-06-13 04:00:00  51.91  52.13  51.89  51.95
497 2024-06-13 04:15:00  51.95  52.39  51.95  52.39
498 2024-06-13 04:30:00  52.39  52.64  51.86  52.14
499 2024-06-13 04:45:00  52.14  52.28  52.04  52.07

[500 rows x 5 columns]


In [5]:
# Example usage
symbol = 'ORDIUSDT'
interval = '15m' 
df = get_binance_ohlc(symbol, interval)
# df = df[200:500]
print(df)


              timestamp   open   high    low  close
0   2024-07-19 22:00:00  37.31  37.47  37.21  37.32
1   2024-07-19 22:15:00  37.32  37.56  36.94  37.05
2   2024-07-19 22:30:00  37.05  37.38  36.94  37.38
3   2024-07-19 22:45:00  37.37  37.37  37.20  37.36
4   2024-07-19 23:00:00  37.37  37.85  37.35  37.70
..                  ...    ...    ...    ...    ...
995 2024-07-30 06:45:00  37.36  37.48  37.28  37.39
996 2024-07-30 07:00:00  37.40  37.54  37.24  37.24
997 2024-07-30 07:15:00  37.26  37.44  36.78  37.17
998 2024-07-30 07:30:00  37.16  37.43  37.12  37.37
999 2024-07-30 07:45:00  37.38  37.48  37.22  37.45

[1000 rows x 5 columns]


In [56]:
def rule(df, tp=1, sl=5, money=1000):
    number_high = 0
    number_low = 0
    cannot_break = 0
    results = []
    df['fluct'] = (df['close'] - df['open']) / df['open'] * 100
    df['amp'] = (df['high'] - df['low']) / df['low'] * 100

    index_condition = []
    for i in range(2, len(df)-1):
        if df['fluct'].iloc[i] < -1 and df['fluct'].iloc[i-1]>0:
            index_condition.append(i+1)
    for i in index_condition:
        a = df['open'].iloc[i]
        high_check = False
        low_check = False
        for j in range(i, len(df)):
            high_check = df['high'].iloc[j] >= (1+sl*0.01) * a 
            low_check = df['low'].iloc[j] <= (1-tp*0.01) * a
            if high_check and low_check:
                print("check lai")
            if high_check:
                # print("high", j-i)
                number_high+=1
                results.append((df.iloc[i]['timestamp'], df.iloc[j]['timestamp'], "high"))
                break
            if low_check:
                number_low+=1
                # print(df.iloc[i]) 
                # print("low", j-i)
                results.append((df.iloc[i]['timestamp'], df.iloc[j]['timestamp'], "low"))
                break
        if not high_check and not low_check:
            cannot_break += 1
            

    print("num points: ", len(index_condition), "number_high/number_low", round(number_high/(number_low+0.1), 3), "cannot_break", cannot_break)
    # print("number_lose:",number_high)
    # print("num win:", number_low)
    return money *   (number_low * tp/100 + number_high * -sl/100),cannot_break, results, index_condition, number_high, number_low


In [54]:
list_day = generate_date_list("2024-05-03", "2024-07-26")

In [57]:
all_reward = 0
all_lose = 0
all_win = 0
all_point = []

for i in range(len(list_day)-1):
    df = get_binance_ohlc_time("ORDIUSDT", "15m", list_day[i]+" 00:00:00", list_day[i+1]+" 00:00:00")
    if i == 0:
        display(df[:1])
    if i == len(list_day) - 2:
        display(df[-1:])
    reward, cb, rs, idcond, high_check, low_check = rule(df)
    all_reward += reward
    all_lose += high_check
    all_win += low_check

    all_point += rs

all_reward, all_lose, all_win

2024-05-03 00:00:00 -> 2024-05-08 00:00:00


,timestamp,open,high,low,close
0,2024-05-03,35.24,35.61,35.24,35.51


num points:  8 number_high/number_low 0.164 cannot_break 1
2024-05-08 00:00:00 -> 2024-05-13 00:00:00
num points:  4 number_high/number_low 0.0 cannot_break 0
2024-05-13 00:00:00 -> 2024-05-18 00:00:00
num points:  15 number_high/number_low 0.36 cannot_break 0
2024-05-18 00:00:00 -> 2024-05-23 00:00:00
num points:  9 number_high/number_low 0.492 cannot_break 0
2024-05-23 00:00:00 -> 2024-05-28 00:00:00
num points:  5 number_high/number_low 0.244 cannot_break 0
2024-05-28 00:00:00 -> 2024-06-02 00:00:00
num points:  12 number_high/number_low 0.33 cannot_break 0
2024-06-02 00:00:00 -> 2024-06-07 00:00:00
num points:  11 number_high/number_low 0.22 cannot_break 0
2024-06-07 00:00:00 -> 2024-06-12 00:00:00
num points:  7 number_high/number_low 0.164 cannot_break 0
2024-06-12 00:00:00 -> 2024-06-17 00:00:00
num points:  4 number_high/number_low 0.0 cannot_break 1
2024-06-17 00:00:00 -> 2024-06-22 00:00:00
num points:  11 number_high/number_low 0.22 cannot_break 0
2024-06-22 00:00:00 -> 2024

,timestamp,open,high,low,close
480,2024-07-22,41.38,41.49,41.07,41.17


num points:  6 number_high/number_low 0.488 cannot_break 0


(-220.00000000000006, 28, 118)

In [58]:
all_point

[(Timestamp('2024-05-03 09:00:00'), Timestamp('2024-05-03 13:15:00'), 'low'),
 (Timestamp('2024-05-03 17:45:00'), Timestamp('2024-05-03 19:45:00'), 'high'),
 (Timestamp('2024-05-04 08:45:00'), Timestamp('2024-05-04 08:45:00'), 'low'),
 (Timestamp('2024-05-04 20:30:00'), Timestamp('2024-05-04 20:30:00'), 'low'),
 (Timestamp('2024-05-06 17:30:00'), Timestamp('2024-05-06 17:30:00'), 'low'),
 (Timestamp('2024-05-07 15:45:00'), Timestamp('2024-05-07 17:00:00'), 'low'),
 (Timestamp('2024-05-07 18:45:00'), Timestamp('2024-05-07 18:45:00'), 'low'),
 (Timestamp('2024-05-08 05:15:00'), Timestamp('2024-05-08 07:15:00'), 'low'),
 (Timestamp('2024-05-08 07:30:00'), Timestamp('2024-05-08 07:30:00'), 'low'),
 (Timestamp('2024-05-08 21:15:00'), Timestamp('2024-05-09 03:00:00'), 'low'),
 (Timestamp('2024-05-12 00:00:00'), Timestamp('2024-05-12 01:00:00'), 'low'),
 (Timestamp('2024-05-14 00:15:00'), Timestamp('2024-05-14 00:15:00'), 'low'),
 (Timestamp('2024-05-14 01:15:00'), Timestamp('2024-05-14 01:15

In [11]:
import time
from datetime import datetime
from binance.spot import Spot
from binance_ft.um_futures import UMFutures
from helper import get_commision, get_precision, get_status_pos, key, secret, xlsx_to_nested_dict

# Initialize the Binance client
api_key="7NvUEUX4tnzOja5KQ99gmUG37DQOV9oelvz1akWAr2Zts9X57djRMwbvfgjQoykp"
api_secret="X9CWCXNsdypjEU8Q0AQaoaqPrcnaX4wpDe5KVxsAfThkVJJAvufiGJ3tb95QqnQC"
# client = UMFutures(key="c21f1bb909318f36de0f915077deadac8322ad7df00c93606970441674c1b39b", secret="be2d7e74239b2e959b3446b880451cbb4dee2e6be9d391c4c55ae7ca0976f403", base_url="https://testnet.binancefuture.com")
client = UMFutures(key=api_key, secret=api_secret)
def get_latest_kline_start_time(symbol, interval):
    klines = client.klines(symbol=symbol, interval=interval, limit=3)
    kline_start_time = int(klines[-1][0])
    return datetime.fromtimestamp(kline_start_time / 1000), klines

    

# Example usage
symbol = 'ORDIUSDT'
interval = '15m'
usdt = 1000
precision_ft = get_precision(symbol, client)
while True:
    kline_start_time, klines = get_latest_kline_start_time(symbol, interval)
    print(kline_start_time)
    next_kline_start_time = kline_start_time + timedelta(minutes=15)
    
    now = datetime.now()
    sleep_duration = (next_kline_start_time - now).total_seconds()    
    if sleep_duration > 0:
        print(f"Sleeping for {sleep_duration} seconds until the next kline starts...")
        time.sleep(sleep_duration)
        current_kline_time, klines = get_latest_kline_start_time(symbol, interval)
        ohlc = []
        for item in klines:
            ohlc.append({
                'timestamp': datetime.fromtimestamp(item[0] / 1000),
                'open': float(item[1]),
                'high': float(item[2]),
                'low': float(item[3]),
                'close': float(item[4])
            })
        df = pd.DataFrame(ohlc)
        df['fluct'] = (df['close'] - df['open']) / df['open'] * 100
        if current_kline_time>kline_start_time:
            print("A new kline has started!")
        else:
            print("In current kline")
            if df['fluct'].iloc[-1] < -0.8 and df['fluct'].iloc[-2] >0:
                order_price = df.iloc[-1]['close']
                high = order_price * 1.05
                low = order_price * 0.99
                print(current_kline_time, order_price, high, low)
                quantity_ft = round((usdt)/order_price, precision_ft)
                print("open")
                # close_future = client.new_order(symbol=symbol, side="SELL", type="MARKET", quantity=quantity_ft)
                while True:
                    askPrice = float(client.book_ticker(symbol)['askPrice'])
                    print("waiting_close...")
                    if askPrice > high or askPrice < low:

                        print(time.strftime("%Y-%m-%d %H:%M:%S"),"close", askPrice, high, low)
                        # close_future = client.new_order(symbol=symbol, side="BUY", type="MARKET", quantity=quantity_ft)
                        break
                    time.sleep(1)
            else:
                display(df)
    else:
        time.sleep(2)


2024-07-30 12:30:00
Sleeping for 115.865656 seconds until the next kline starts...
In current kline


,timestamp,open,high,low,close,fluct
0,2024-07-30 12:00:00,37.629,37.629,37.494,37.529,-0.265752
1,2024-07-30 12:15:00,37.528,37.550,37.361,37.441,-0.231827
2,2024-07-30 12:30:00,37.438,37.532,37.387,37.485,0.125541


2024-07-30 12:30:00
2024-07-30 12:30:00
2024-07-30 12:45:00
Sleeping for 895.530002 seconds until the next kline starts...
In current kline


,timestamp,open,high,low,close,fluct
0,2024-07-30 12:15:00,37.528,37.550,37.361,37.441,-0.231827
1,2024-07-30 12:30:00,37.438,37.532,37.387,37.485,0.125541
2,2024-07-30 12:45:00,37.485,37.614,37.468,37.549,0.170735


2024-07-30 12:45:00
2024-07-30 13:00:00
Sleeping for 897.268858 seconds until the next kline starts...
In current kline


...

,timestamp,open,high,low,close,fluct
0,2024-07-30 21:15:00,37.437,37.557,37.372,37.435,-0.005342
1,2024-07-30 21:30:00,37.436,37.591,37.374,37.391,-0.120205
2,2024-07-30 21:45:00,37.394,37.652,37.237,37.434,0.106969


2024-07-30 21:45:00
2024-07-30 21:45:00
2024-07-30 22:00:00
Sleeping for 895.180151 seconds until the next kline starts...
In current kline


,timestamp,open,high,low,close,fluct
0,2024-07-30 21:30:00,37.436,37.591,37.374,37.391,-0.120205
1,2024-07-30 21:45:00,37.394,37.652,37.237,37.434,0.106969
2,2024-07-30 22:00:00,37.434,37.787,37.289,37.729,0.788054


2024-07-30 22:00:00
2024-07-30 22:00:00
2024-07-30 22:15:00
Sleeping for 895.207479 seconds until the next kline starts...


KeyboardInterrupt: 

In [480]:
df

,timestamp,open,high,low,close,fluct
0,2024-07-30 00:02:00,66550.0,66650.0,66365.8,66630.1,0.120361
1,2024-07-30 00:03:00,66630.0,66759.7,66573.0,66759.6,0.194507
2,2024-07-30 00:04:00,66759.6,66850.0,66721.8,66809.9,0.075345


In [ ]:
import mplfinance as mpf

mpf.plot(df, type='candle', style='charles', title='BTC/USDT OHLC', ylabel='Price (USD)')
